In [1]:
import torch
import time
import subprocess
import threading
import numpy as np

# GPU 상태 모니터링 함수
def monitor_gpu():
    while monitoring:
        try:
            # Windows에서 실행
            subprocess.run("nvidia-smi", shell=True)
        except:
            print("nvidia-smi 명령어를 실행할 수 없습니다.")
        time.sleep(3)  # 3초마다 업데이트

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU 사용 가능: {torch.cuda.get_device_name(0)}")
    print(f"CUDA 버전: {torch.version.cuda}")
else:
    device = torch.device("cpu")
    print("GPU를 사용할 수 없습니다. CPU를 사용합니다.")
    
# GPU 모니터링 시작
monitoring = True
monitor_thread = threading.Thread(target=monitor_gpu)
monitor_thread.daemon = True
monitor_thread.start()

# 테스트 지속 시간 설정 (초)
test_duration = 60  # 1분 동안 테스트 (원하는 시간으로 조정 가능)
print(f"\n{test_duration}초 동안 GPU 부하 테스트를 실행합니다...")

# 시작 시간 기록
start_time = time.time()
end_time = start_time + test_duration

# 다양한 크기의 텐서로 테스트
tensor_sizes = [5000, 8000, 10000, 12000]
current_size_index = 0

# 테스트 중 GPU 사용률 기록
gpu_utils = []

print("\nGPU 부하 테스트 시작...")
iteration = 0

try:
    while time.time() < end_time:
        # 현재 텐서 크기 선택 (주기적으로 변경)
        tensor_size = tensor_sizes[current_size_index]
        current_size_index = (current_size_index + 1) % len(tensor_sizes)
        
        # 남은 시간 계산
        remaining_time = max(0, int(end_time - time.time()))
        
        # 진행 상황 출력
        print(f"\n반복 {iteration+1}: 텐서 크기 {tensor_size}x{tensor_size} 사용 중 (남은 시간: {remaining_time}초)")
        
        # 큰 텐서 생성
        a = torch.randn(tensor_size, tensor_size, device=device)
        b = torch.randn(tensor_size, tensor_size, device=device)
        
        # 행렬 곱셈 수행
        print(f"행렬 곱셈 수행 중...")
        c = torch.matmul(a, b)
        
        # 결과 일부 출력
        print(f"결과 샘플: {c[0, 0].item():.4f}")
        
        # 추가 연산 수행 (GPU 부하 유지)
        d = torch.sin(a) + torch.cos(b)
        e = torch.exp(torch.abs(d) * 0.01)  # 값이 너무 커지지 않도록 조정
        
        # 메모리 정보 출력
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        print(f"GPU 메모리: 할당됨 {allocated:.2f} GB, 예약됨 {reserved:.2f} GB")
        
        # 메모리 비우기
        del a, b, c, d, e
        torch.cuda.empty_cache()
        
        # 잠시 대기 (너무 빨리 끝나지 않도록)
        time.sleep(1)
        
        iteration += 1

except KeyboardInterrupt:
    print("\n사용자에 의해 테스트가 중단되었습니다.")

finally:
    # 테스트 종료
    actual_duration = time.time() - start_time
    print(f"\nGPU 부하 테스트 완료! 실제 소요 시간: {actual_duration:.2f}초")
    
    # 최종 GPU 정보 출력
    print("\nGPU 메모리 사용량 정보:")
    print(f"최대 메모리 사용량: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GB")
    
    # 모니터링 종료
    monitoring = False
    print("\n테스트가 완료되었습니다. Ctrl+C를 눌러 프로그램을 종료하세요.")

GPU 사용 가능: NVIDIA GeForce RTX 4070 Ti
CUDA 버전: 11.8

60초 동안 GPU 부하 테스트를 실행합니다...

GPU 부하 테스트 시작...

반복 1: 텐서 크기 5000x5000 사용 중 (남은 시간: 60초)
행렬 곱셈 수행 중...
결과 샘플: 122.5649
GPU 메모리: 할당됨 0.48 GB, 예약됨 0.58 GB

반복 2: 텐서 크기 8000x8000 사용 중 (남은 시간: 58초)
행렬 곱셈 수행 중...
결과 샘플: -68.3077
GPU 메모리: 할당됨 1.20 GB, 예약됨 1.46 GB

반복 3: 텐서 크기 10000x10000 사용 중 (남은 시간: 57초)
행렬 곱셈 수행 중...
결과 샘플: -131.5038
GPU 메모리: 할당됨 1.87 GB, 예약됨 2.26 GB

반복 4: 텐서 크기 12000x12000 사용 중 (남은 시간: 56초)
행렬 곱셈 수행 중...
결과 샘플: -142.6721
GPU 메모리: 할당됨 2.69 GB, 예약됨 3.24 GB

반복 5: 텐서 크기 5000x5000 사용 중 (남은 시간: 55초)
행렬 곱셈 수행 중...
결과 샘플: -18.9555
GPU 메모리: 할당됨 0.48 GB, 예약됨 0.58 GB

반복 6: 텐서 크기 8000x8000 사용 중 (남은 시간: 54초)
행렬 곱셈 수행 중...
결과 샘플: 61.9152
GPU 메모리: 할당됨 1.20 GB, 예약됨 1.46 GB

반복 7: 텐서 크기 10000x10000 사용 중 (남은 시간: 53초)
행렬 곱셈 수행 중...
결과 샘플: 131.8719
GPU 메모리: 할당됨 1.87 GB, 예약됨 2.26 GB

반복 8: 텐서 크기 12000x12000 사용 중 (남은 시간: 52초)
행렬 곱셈 수행 중...
결과 샘플: -30.3713
GPU 메모리: 할당됨 2.69 GB, 예약됨 3.24 GB

반복 9: 텐서 크기 5000x5000 사용 중 (남은 시간: 50초)
행렬 곱셈 수행 중.

In [2]:
import torch
import subprocess
import os
from ultralytics import YOLO

# GPU 상태 확인
print(f"GPU 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 모델: {torch.cuda.get_device_name(0)}")
    print(f"CUDA 버전: {torch.version.cuda}")

# YOLOv8n 모델 로드
print("\nYOLOv8n 모델 로드 중...")
model = YOLO("yolov8n.pt")
print("모델 로드 완료!")

# 학습 설정
print("\n학습 설정:")
epochs = 30  # 테스트용 에포크 수
batch_size = 16  # 명시적 배치 크기 설정
imgsz = 640  # 이미지 크기
data = "coco8.yaml"  # Ultralytics 내장 샘플 데이터셋

print(f"- 에포크: {epochs}")
print(f"- 배치 크기: {batch_size}")
print(f"- 이미지 크기: {imgsz}")
print(f"- 데이터 설정: {data} (내장 샘플 데이터셋)")

# GPU 사용량 모니터링 시작
print("\nGPU 사용량 모니터링 시작...")
try:
    # Windows에서는 별도 창에서 실행
    subprocess.Popen("start cmd /k nvidia-smi -l 5", shell=True)
except:
    print("nvidia-smi 명령어를 실행할 수 없습니다.")

# 학습 시작
print("\n학습 시작...")
try:
    results = model.train(
        data=data,  # 내장 샘플 데이터셋 사용
        epochs=epochs,
        imgsz=imgsz,
        batch=batch_size,
        name="yolov8n_coco8_test",
        verbose=True,
        device=0,
        workers=8,
        exist_ok=True,
        pretrained=True,
        patience=0  # 테스트용으로 early stopping 비활성화
    )
    
    print("\n학습 완료!")
    print(f"결과 저장 경로: {os.path.join('runs', 'detect', 'yolov8n_coco8_test')}")
    
    # 학습 결과 요약
    print("\n학습 결과 요약:")
    if hasattr(results, 'results_dict'):
        print(f"- 최종 mAP50: {results.results_dict.get('metrics/mAP50(B)', 'N/A')}")
        print(f"- 최종 mAP50-95: {results.results_dict.get('metrics/mAP50-95(B)', 'N/A')}")
        print(f"- 최종 손실: {results.results_dict.get('metrics/loss(B)', 'N/A')}")
    
    # 검증 실행
    print("\n검증 실행 중...")
    val_results = model.val()
    print("검증 완료!")
    
except Exception as e:
    print(f"\n학습 중 오류 발생: {e}")

finally:
    # GPU 정보 출력
    if torch.cuda.is_available():
        print("\nGPU 메모리 사용량:")
        print(f"할당된 메모리: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
        print(f"최대 메모리 사용량: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GB")
    
    print("\n테스트 완료!")

GPU 사용 가능: True
GPU 모델: NVIDIA GeForce RTX 4070 Ti
CUDA 버전: 11.8

YOLOv8n 모델 로드 중...
모델 로드 완료!

학습 설정:
- 에포크: 30
- 배치 크기: 16
- 이미지 크기: 640
- 데이터 설정: coco8.yaml (내장 샘플 데이터셋)

GPU 사용량 모니터링 시작...

학습 시작...
New https://pypi.org/project/ultralytics/8.3.98 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.86  Python-3.9.20 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=coco8.yaml, epochs=30, time=None, patience=0, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=yolov8n_coco8_test, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, ha

c:\Users\dongi\anaconda3\envs\vision-gpu\lib\site-packages\ultralytics\utils\torch_utils.py:262: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\Context.cpp:208.)
  fusedconv.weight.copy_(torch.mm(w_bn, w_conv).view(fusedconv.weight.shape))
c:\Users\dongi\anaconda3\envs\vision-gpu\lib\site-packages\ultralytics\utils\torch_utils.py:267: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` 

AMP: checks passed 


train: Scanning C:\Users\dongi\git\model-conversion-2-spring-2025\CV\object_detection\datasets\coco8\labels\train.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
val: Scanning C:\Users\dongi\git\model-conversion-2-spring-2025\CV\object_detection\datasets\coco8\labels\val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]


Plotting labels to runs\detect\yolov8n_coco8_test\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\yolov8n_coco8_test
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


30 epochs completed in 0.005 hours.
Optimizer stripped from runs\detect\yolov8n_coco8_test\weights\last.pt, 6.5MB
Optimizer stripped from runs\detect\yolov8n_coco8_test\weights\best.pt, 6.5MB

Validating runs\detect\yolov8n_coco8_test\weights\best.pt...
Ultralytics 8.3.86  Python-3.9.20 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
Model summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


c:\Users\dongi\anaconda3\envs\vision-gpu\lib\site-packages\ultralytics\utils\torch_utils.py:262: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\Context.cpp:208.)
  fusedconv.weight.copy_(torch.mm(w_bn, w_conv).view(fusedconv.weight.shape))
c:\Users\dongi\anaconda3\envs\vision-gpu\lib\site-packages\ultralytics\utils\torch_utils.py:267: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` 

                   all          4         17      0.599      0.898      0.888      0.623
                person          3         10      0.646        0.5       0.52      0.286
                   dog          1          1      0.319          1      0.995      0.597
                 horse          1          2        0.7          1      0.995      0.648
              elephant          1          2       0.63      0.891      0.828      0.319
              umbrella          1          1      0.542          1      0.995      0.995
          potted plant          1          1       0.76          1      0.995      0.895
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\yolov8n_coco8_test

학습 완료!
결과 저장 경로: runs\detect\yolov8n_coco8_test

학습 결과 요약:
- 최종 mAP50: 0.8880077331262538
- 최종 mAP50-95: 0.6233568480091135
- 최종 손실: N/A

검증 실행 중...
Ultralytics 8.3.86  Python-3.9.20 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
Model

val: Scanning C:\Users\dongi\git\model-conversion-2-spring-2025\CV\object_detection\datasets\coco8\labels\val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.21s/it]


                   all          4         17      0.599      0.897      0.888      0.625
                person          3         10      0.646        0.5      0.519      0.296
                   dog          1          1       0.32          1      0.995      0.597
                 horse          1          2        0.7          1      0.995      0.648
              elephant          1          2      0.628      0.883      0.828      0.319
              umbrella          1          1      0.542          1      0.995      0.995
          potted plant          1          1      0.761          1      0.995      0.895
Speed: 1.5ms preprocess, 21.0ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs\detect\yolov8n_coco8_test
검증 완료!

GPU 메모리 사용량:
할당된 메모리: 0.08 GB
최대 메모리 사용량: 3.23 GB

테스트 완료!
